<a href="https://colab.research.google.com/github/matinmazid/York_ML_certificate/blob/master/M3_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#connecct to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec #Word2Vec is mostly used for huge datasets
pd.options.display.max_colwidth = 200
%matplotlib inline

In [5]:
import pandas as pd
# Path to the folder where the CSV files are saved
folder_path = '/content/drive/MyDrive/ColabNotebooks/ML1010/Dataset/'
# Read the CSV files and assign them to variables
X_train_bow = pd.read_csv(f'{folder_path}X_train_bow.csv').values
X_test_bow = pd.read_csv(f'{folder_path}X_test_bow.csv').values
X_train_tfidf = pd.read_csv(f'{folder_path}X_train_tfidf.csv').values
X_test_tfidf = pd.read_csv(f'{folder_path}X_test_tfidf.csv').values
X_train_w2v = pd.read_csv(f'{folder_path}X_train_w2v.csv').values
X_test_w2v = pd.read_csv(f'{folder_path}X_test_w2v.csv').values
X_train_glove = pd.read_csv(f'{folder_path}X_train_glove.csv').values
X_test_glove = pd.read_csv(f'{folder_path}X_test_glove.csv').values
X_train_bert = pd.read_csv(f'{folder_path}X_train_bert.csv').values
X_test_bert = pd.read_csv(f'{folder_path}X_test_bert.csv').values
# Print the shapes of the variables to confirm they are loaded correctly
print(f'X_train_bow shape: {X_train_bow.shape}')
print(f'X_test_bow shape: {X_test_bow.shape}')
print(f'X_train_tfidf shape: {X_train_tfidf.shape}')
print(f'X_test_tfidf shape: {X_test_tfidf.shape}')
print(f'X_train_w2v shape: {X_train_w2v.shape}')
print(f'X_test_w2v shape: {X_test_w2v.shape}')
print(f'X_train_glove shape: {X_train_glove.shape}')
print(f'X_test_glove shape: {X_test_glove.shape}')
print(f'X_train_bert shape: {X_train_bert.shape}')
print(f'X_test_bert shape: {X_test_bert.shape}')

X_train_bow shape: (35000, 5000)
X_test_bow shape: (15000, 5000)
X_train_tfidf shape: (35000, 5000)
X_test_tfidf shape: (15000, 5000)
X_train_w2v shape: (35000, 100)
X_test_w2v shape: (15000, 100)
X_train_glove shape: (35000, 100)
X_test_glove shape: (15000, 100)
X_train_bert shape: (35000, 768)
X_test_bert shape: (15000, 768)


In [7]:
df_train = pd.read_csv(f'{folder_path}cleaned_train_df.csv')
df_test = pd.read_csv(f'/{folder_path}cleaned_test_df.csv')
print(df_test.shape)
df_test.head()

(15000, 2)


,clean_review,sentiment
0,really liked summerslam due look arena curtain look overall interesting reason anyways could one best summerslam ever wwf lex luger main event yokozuna time ok huge fat man v strong man glad time ...,1
1,many television show appeal quite many different kind fan like farscape know youngster year old fan male female many different country think adore v miniseries elements find almost every show v ch...,1
2,film quickly get major chase scene ever increase destruction first really bad thing guy hijack steven seagal would beat pulp seagal driving probably would end whole premise movie seem like decide ...,0
3,jane austen would definitely approve one gwyneth paltrow awesome job capture attitude emma funny without excessively silly yet elegant put convince british accent british maybe best judge fool als...,1
4,expectation somewhat high go see movie think steve carell could wrong come great movie like anchorman year old virgin little miss sunshine boy wrong start right movie certain point steve carell al...,0


In [12]:
y_train = df_train['sentiment']
X_test = df_test['clean_review']
y_test = df_test['sentiment']

#Model Selecting

In [9]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,roc_auc_score,auc
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
models=[]
# Define the baseline model
LR=LogisticRegression(max_iter=1000)
# Define other models
NB=MultinomialNB()
KN=KNeighborsClassifier()
DT=DecisionTreeClassifier(criterion='entropy')
RF=RandomForestClassifier(n_estimators=100, criterion='gini')
models.append(LR)
models.append(NB)
models.append(KN)
models.append(DT)
models.append(RF)
# Initialize an empty list to store results
results = []
# Function to train and evaluate, and store results in a list
def train_evaluate(model, X_train, X_test, y_train, y_test, method_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    # Get classification report metrics
    report = classification_report(y_test, y_pred, output_dict=True)
    # Extract precision, recall, and f1-score for each class
    precision = report['accuracy']  # Overall accuracy
    recall = np.mean([report[str(i)]['recall'] for i in range(2)])
    f1 = np.mean([report[str(i)]['f1-score'] for i in range(2)])
    # Append the results to the list
    results.append({
        'Method': method_name,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1
    })

for name,model in models:
  print("=======================")
  print("Model Name: ",name)
  # Train and evaluate for each embedding and store the results
  # Model with BoW
  train_evaluate(model, X_train_bow, X_test_bow, y_train, y_test, 'BoW')
  # Model with TF-IDF
  train_evaluate(model, X_train_tfidf, X_test_tfidf, y_train, y_test, 'TF-IDF')
  # Model with Word2Vec
  train_evaluate(model, X_train_w2v, X_test_w2v, y_train, y_test, 'Word2Vec')
  # Model with GloVe
  train_evaluate(model, X_train_glove, X_test_glove, y_train, y_test, 'GloVe')
  # Model with BERT
  train_evaluate(model, X_train_bert, X_test_bert, y_train, y_test, 'BERT')
  results_df_tfidf = pd.DataFrame(results)
  # Display the comparison table
  results_df_tfidf

TypeError: cannot unpack non-iterable LogisticRegression object

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

Matin look here

In [19]:
!pip install chart_studio
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 1.9 MB/s eta 0:00:00


In [44]:

def myParameterExperiment(treeCount,criterion, Xtrain,y_train,XTest):
  RF=RandomForestClassifier(n_estimators=treeCount, criterion=criterion)
  RF.fit(Xtrain, y_train)
#train_evaluate(model, X_train_bow, X_test_bow, y_train, y_test, 'BoW')
#def train_evaluate(model, X_train, X_test, y_train, y_test, method_name):
  RF.fit(Xtrain, y_train)
  y_pred = RF.predict(XTest)
    # Get classification report metrics
  report = classification_report(y_test, y_pred, output_dict=True)
    # Extract precision, recall, and f1-score for each class
  precision = report['accuracy']  # Overall accuracy
  recall = np.mean([report[str(i)]['recall'] for i in range(2)])
  f1 = np.mean([report[str(i)]['f1-score'] for i in range(2)])
    # Append the results to the list
  print(" found it" ,precision,recall,f1,criterion)
  return [precision,recall,f1]

In [ ]:
import numpy as np
criterion_list=["gini","entropy", "log_loss"]
# estrimtors_list=[100,200,300,400,500]
estrimtors_list=[5,10]

hyperparamter_combo=len(criterion_list)*len(estrimtors_list)

precisionList=[]
recallList=[]
f1List=[]
for criterion in criterion_list:
  for n_estimators in estrimtors_list:
    assessment=myParameterExperiment(n_estimators,criterion,X_train_bow,y_train,X_test_bow)
    precisionList.append(assessment[0])
    recallList.append(assessment[1])
    f1List.append(assessment[2])

mydictionary={
               "n_estimators":estrimtors_list+[None]*(hyperparamter_combo-len(estrimtors_list)),
               "criterion":criterion_list+[None]*(hyperparamter_combo-len(criterion_list)),
               "precision":precisionList,
               "recall":recallList,
               "f1":f1List}
resultDF = pd.DataFrame(mydictionary)
resultDF

# import pandas as pd

# data = [{"a": "ss", "b": 1}, {"a": "yy", "b": 3}]
# df = pd.DataFrame(data)
#mydictionary

 found it 0.751 0.751116054938154 0.7509998129731928 gini


In [ ]:
from plotly.offline import iplot
import chart_studio.plotly as py
import plotly.graph_objects as go

import numpy as np

criterion_list=["gini","entropy", "log_loss"]
estrimtors_list=[100,200,300,400,500]

x=pd.DataFrame(criterion_list,columns=['criterion'])
y=pd.DataFrame(estrimtors_list,columns=['Numbers'])
data = [go.Scatter(x=x["criterion"],
            y=y["Numbers"])]

iplot(data, filename='jupyter-basic_bar')

fig = go.Figure(data=data)
# iplot(fig, filename='jupyter-parametric_plot')
data

Matin dont look any further

In [ ]:
from sklearn.metrics import roc_auc_score
from itertools import cycle
def roc_curve_plot(models,X_test,y_test):
  for name,model in models:
    print("=======================")
    print("Model Name: ",name)
    model_prob = model.predict_proba(X_test)

    # roc curve for models
    fpr1, tpr1, thresh1 = roc_curve(y_test, model_prob[:,1], pos_label=1)

    # roc curve for tpr = fpr
    random_probs = [0 for i in range(len(y_test))]
    p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)

    # auc scores
    auc_model = roc_auc_score(y_test, model_prob[:,1])
    print(auc_model)

    # plot roc curves
    colors = cycle(['red','green','navy', 'aqua', 'darkorange', 'cornflowerblue'])
    plt.plot(fpr1, tpr1, linestyle='--',color=colors, label=name + ' auc=' + str(auc_model))
    plt.plot(p_fpr, p_tpr, linestyle='--', color='deeppink')

  # title
  plt.title('ROC curve')
  # x label
  plt.xlabel('False Positive Rate')
  # y label
  plt.ylabel('True Positive rate')

  plt.legend(loc='best')
  plt.savefig('ROC',dpi=800)
  plt.show();

# default test dataset
roc_curve_plot(models,X_test,y_test)

# Model Optimization